In [10]:
import pandas as pd
import os

training_data = pd.read_csv("feature_trained.csv")
testing_data = pd.read_csv("test_20.csv")

In [11]:
training_data.head()

,http_time_avg,ttl_avg,packet_inter_arrivel_B_firstQ,packet_inter_arrivel_A_sum,packet_inter_arrivel_B_min,device_category
0,0.000000,59.172414,0.022090,493.271226,0.000000,0
1,0.003083,64.000000,0.000376,0.038255,0.000041,0
2,0.004329,64.000000,0.000371,0.037288,0.000041,0
3,0.003404,64.000000,0.000376,0.032050,0.000040,0
4,0.004324,64.000000,0.000370,0.028010,0.000040,0


In [12]:
from sklearn.model_selection import train_test_split

train, val = train, val = train_test_split(training_data, test_size=0.20) #random_state=12)

X_train = train.drop(columns=["device_category"])
y_train = train["device_category"].values

x_val = val.drop(columns=["device_category"])
y_val = val["device_category"].values

In [13]:
X_train.shape

(800, 5)

In [14]:
x_val.shape

(200, 5)

In [15]:
X_train

,http_time_avg,ttl_avg,packet_inter_arrivel_B_firstQ,packet_inter_arrivel_A_sum,packet_inter_arrivel_B_min
722,0.0,51.608696,7.480380e-04,1.826781,1.310000e-05
491,0.0,54.434783,9.540000e-07,1.891229,0.000000e+00
448,0.0,54.434783,9.540000e-07,2.306481,0.000000e+00
874,0.0,59.086957,1.080036e-03,414.445139,6.100000e-05
271,0.0,255.000000,0.000000e+00,1.498904,0.000000e+00
...,...,...,...,...,...
32,0.0,54.380952,3.232960e-04,1.108056,9.540000e-07
994,0.0,255.000000,0.000000e+00,1.508748,0.000000e+00
702,0.0,53.315789,3.050000e-05,1.783076,2.000000e-05
764,0.0,52.230769,2.000000e-05,1.972186,9.060000e-06


In [16]:
x_val

,http_time_avg,ttl_avg,packet_inter_arrivel_B_firstQ,packet_inter_arrivel_A_sum,packet_inter_arrivel_B_min
672,0.0,255.000000,0.000000,1.498901,0.000000
898,0.0,66.360000,0.034723,241.395995,0.000111
780,0.0,52.371429,0.005871,3.120873,0.000000
406,0.0,55.000000,0.000002,2.410747,0.000000
514,0.0,161.812500,0.067095,4.618842,0.000269
...,...,...,...,...,...
990,0.0,255.000000,0.000000,1.499551,0.000000
765,0.0,52.088235,0.000292,2.844735,0.000012
935,0.0,255.000000,0.000000,1700.208887,0.000000
228,0.0,255.000000,0.000000,1.498290,0.000000


In [17]:
from xgboost import XGBClassifier

xgb_params = {
    'n_estimators': 50,
    'random_state': 11

}
#insert code here for modelling

xgb = XGBClassifier(**xgb_params)


xgb.fit(X_train, y_train)
xgb_preds_train = xgb.predict(X_train)
xgb_preds_val = xgb.predict(x_val)

In [18]:
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report

#XGB Classifier
xgb_predicted = xgb.predict(x_val)
xgb_conf_matrix = confusion_matrix(y_val, xgb_predicted)
xgb_acc_score = accuracy_score(y_val, xgb_predicted)
print("confusion matrix")
print(xgb_conf_matrix)
print("\n")
print("Accuracy of XGB:",xgb_acc_score*100,'\n')
print(classification_report(y_val, xgb_predicted))

xgb_preds_train = xgb.predict(X_train)
xgb_preds_val = xgb.predict(x_val)

print('XGB:\n> Accuracy on training data = {:.4f}\n> Accuracy on validation data = {:.4f}'.format(
    accuracy_score(y_true=y_train, y_pred=xgb_preds_train),
    accuracy_score(y_true=y_val, y_pred=xgb_preds_val)
))

confusion matrix
[[15  0  0  0  0  0  0  0  2  0]
 [ 0 14  0  0  1  0  0  0  0  0]
 [ 0  0 10  0  0  0  7  0  0  4]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  3  0  0 21  0  0  0  0  0]
 [ 0  0  0  1  1 29  0  0  0  0]
 [ 0  0  8  0  0  0 15  0  0  3]
 [ 0  0  0  0  0  0  0 13  0  0]
 [ 2  0  0  0  0  0  0  0 16  0]
 [ 0  0  2  0  0  0  4  0  0 11]]


Accuracy of XGB: 81.0 

              precision    recall  f1-score   support

           0       0.88      0.88      0.88        17
           1       0.82      0.93      0.87        15
           2       0.50      0.48      0.49        21
           3       0.95      1.00      0.97        18
           4       0.91      0.88      0.89        24
           5       1.00      0.94      0.97        31
           6       0.58      0.58      0.58        26
           7       1.00      1.00      1.00        13
           8       0.89      0.89      0.89        18
           9       0.61      0.65      0.63        17

    accuracy                  

In [19]:
val_df = pd.read_csv("test_20.csv")
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   http_time_avg                  201 non-null    float64
 1   ttl_avg                        201 non-null    float64
 2   packet_inter_arrivel_B_firstQ  201 non-null    float64
 3   packet_inter_arrivel_A_sum     201 non-null    float64
 4   packet_inter_arrivel_B_min     201 non-null    float64
 5   device_category                201 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 9.5 KB


In [20]:
test_data = val_df.drop(columns=['device_category'])
answer_data = val_df['device_category'].values

In [21]:
import pickle

pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(xgb, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = pickle_model.score(test_data, answer_data)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(test_data)

Test score: 55.72 %


In [22]:
xgb.score(test_data, answer_data)

0.5572139303482587